In [7]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import time

print("--- Starting F1 Lap Time Model Training (v4, Final) ---")

# --- 1. Define Features & Target ---
TARGET = 'Lap_Time_Seconds' 

NUMERIC_FEATURES = [
    'Len_Circuit_inkm', 'Laps', 'Start_Position', 'Formula_Avg_Speed_kmh',
    'Humidity_%', 'Champ_Points', 'Champ_Position', 'race_year',
    'seq', 'position', 'points', 'Corners_in_Lap', 'Tire_Degradation_Factor_per_Lap',
    'Pit_Stop_Duration_Seconds', 'Ambient_Temperature_Celsius', 
    'Track_Temperature_Celsius', 'air', 'ground', 'starts', 'finishes',
    'with_points', 'podiums', 'wins'
]

CATEGORICAL_FEATURES = [
    'Rider_ID', 'Formula_category_x', 'Formula_Track_Condition',
    'Tire_Compound', 'Penalty', 'Session', 'Formula_shortname',
    'circuit_name', 'weather', 'track'
]

# --- 2. Load Your Training Data (Memory Efficient) ---
try:
    print("Loading 'train.csv'...")
    # Set dtypes for categories AND numeric columns that might be objects
    # This is a safety measure
    dtype_map = {col: 'category' for col in CATEGORICAL_FEATURES}
    for col in NUMERIC_FEATURES:
        dtype_map[col] = 'float64' # Load numeric columns as floats
        
    df_raw = pd.read_csv('train.csv', dtype=dtype_map)
    print(f"Loaded 'train.csv' with {df_raw.shape[0]} rows.")
except FileNotFoundError:
    print("FATAL ERROR: 'train.csv' not found.")
    exit()
except ValueError as e:
    print(f"Warning: Could not load all dtypes automatically: {e}")
    print("Loading with default dtypes and will convert later.")
    df_raw = pd.read_csv('train.csv')


# --- 3. Feature Engineering Function ---
def engineer_features(df):
    """Creates new features."""
    print("Engineering features...")
    
    if 'weather' in df.columns:
        df['is_raining'] = df['weather'].astype(str).str.contains('Rain', case=False).fillna(0)
        if 'is_raining' not in NUMERIC_FEATURES:
            NUMERIC_FEATURES.append('is_raining')

    if 'Track_Temperature_Celsius' in df.columns and 'Ambient_Temperature_Celsius' in df.columns:
        # Ensure columns are numeric before subtraction
        df['Track_Temperature_Celsius'] = pd.to_numeric(df['Track_Temperature_Celsius'], errors='coerce')
        df['Ambient_Temperature_Celsius'] = pd.to_numeric(df['Ambient_Temperature_Celsius'], errors='coerce')
        df['temp_diff'] = df['Track_Temperature_Celsius'] - df['Ambient_Temperature_Celsius']
        if 'temp_diff' not in NUMERIC_FEATURES:
            NUMERIC_FEATURES.append('temp_diff')
            
    return df

# --- 4. Apply Feature Engineering ---
df = engineer_features(df_raw.copy())

# --- Data Type Conversion (Critical) ---
# Ensure all numeric features are actually numeric (float)
# and categorical features are 'category'
print("Converting data types...")
for col in NUMERIC_FEATURES:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce') # Force to numeric
        
for col in CATEGORICAL_FEATURES:
    if col in df.columns:
        df[col] = df[col].astype('category')

# Safety checks
NUMERIC_FEATURES = [col for col in NUMERIC_FEATURES if col in df.columns]
CATEGORICAL_FEATURES = [col for col in CATEGORICAL_FEATURES if col in df.columns]

if TARGET not in df.columns:
    print(f"FATAL ERROR: Target column '{TARGET}' not found.")
    exit()

# --- 5. Create Data Splits ---
X = df.drop(TARGET, axis=1)
y = df[TARGET]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training on {len(X_train)} samples, validating on {len(X_val)} samples.")

# --- 6. Set up Preprocessing (FIXED) ---
# This pipeline will *only* process numeric features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Create the ColumnTransformer to apply the numeric pipeline
# We set remainder='drop' because we will handle categorical columns manually
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, NUMERIC_FEATURES)
    ],
    remainder='drop' # <-- This is the key change
)

# --- 7. Apply Preprocessing Manually (FIXED) ---
print("Applying preprocessing...")

# Fit the preprocessor on the training data's numeric columns
preprocessor.fit(X_train[NUMERIC_FEATURES])

# --- New function to apply transform and rebuild DataFrame ---
def preprocess_data(df, processor):
    # 1. Transform the numeric features
    scaled_numeric_data = processor.transform(df[NUMERIC_FEATURES])
    
    # 2. Put the scaled data back into a DataFrame
    df_numeric_scaled = pd.DataFrame(scaled_numeric_data, columns=NUMERIC_FEATURES, index=df.index)
    
    # 3. Get the original categorical features (and ensure correct type)
    df_categorical = df[CATEGORICAL_FEATURES].astype('category')
    
    # 4. Concatenate the scaled numeric and original categorical features
    df_final = pd.concat([df_numeric_scaled, df_categorical], axis=1)
    
    return df_final

# Apply the new preprocessing to our train and validation sets
X_train_final = preprocess_data(X_train, preprocessor)
X_val_final = preprocess_data(X_val, preprocessor)

print("Preprocessing complete. Final training data types:")
print(X_train_final.info())

# --- 8. Define the XGBoost Model (GPU Enabled) ---
# --- 8. Define the XGBoost Model (CPU Fallback) ---
print("Defining model to run on CPU (GPU not found)...")
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    early_stopping_rounds=50,
    
    # --- THIS IS THE FIX ---
    tree_method='hist',      # Use the fast CPU method
    # ----------------------
    
    enable_categorical=True      # Natively handle categories
)

# --- 9. Train the Model (No Pipeline) ---
print("\n🏎️  Starting model training...")
start_time = time.time()

# Fit the model directly on the preprocessed DataFrames
model.fit(
    X_train_final, 
    y_train,
    eval_set=[(X_val_final, y_val)], 
    verbose=False 
)

end_time = time.time()
print(f"✅ Training complete in {end_time - start_time:.2f} seconds.")

# --- 10. Evaluate Your New Model ---
y_pred = model.predict(X_val_final)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

print(f"\n--- 🏁 Model Evaluation ---")
print(f"Validation RMSE: {rmse:.4f} seconds")

# --- 11. Load 'test.csv' for Final Predictions ---
print("\nLoading 'test.csv' for final predictions...")
try:
    df_test_raw = pd.read_csv('test.csv')
    submission_ids = df_test_raw['id']
except FileNotFoundError:
    print("FATAL ERROR: 'test.csv' not found.")
    exit()

# --- 12. Apply the SAME Transformations to Test Data ---
df_test_engineered = engineer_features(df_test_raw.copy())

# Ensure dtypes are correct *before* preprocessing
for col in NUMERIC_FEATURES:
    if col in df_test_engineered.columns:
        df_test_engineered[col] = pd.to_numeric(df_test_engineered[col], errors='coerce')
for col in CATEGORICAL_FEATURES:
    if col in df_test_engineered.columns:
        df_test_engineered[col] = df_test_engineered[col].astype('category')
        
df_test_final = preprocess_data(df_test_engineered, preprocessor)

# --- 13. Make Final Predictions ---
print("Making predictions on test data...")
final_predictions = model.predict(df_test_final)

# --- 14. Create Submission File ---
print("Creating 'submission.csv'...")
submission_df = pd.DataFrame({
    'id': submission_ids,
    'Lap_Time_Seconds': final_predictions
})
submission_df['Lap_Time_Seconds'] = submission_df['Lap_Time_Seconds'].apply(lambda x: max(0, x))
submission_df.to_csv('submission.csv', index=False)

print("\n✅ Done! Your 'submission.csv' file is ready.")

--- Starting F1 Lap Time Model Training (v4, Final) ---
Loading 'train.csv'...
Loaded 'train.csv' with 734002 rows.
Engineering features...
Converting data types...
Training on 587201 samples, validating on 146801 samples.
Applying preprocessing...
Preprocessing complete. Final training data types:
<class 'pandas.core.frame.DataFrame'>
Index: 587201 entries, 391931 to 121958
Data columns (total 35 columns):
 #   Column                           Non-Null Count   Dtype   
---  ------                           --------------   -----   
 0   Len_Circuit_inkm                 587201 non-null  float64 
 1   Laps                             587201 non-null  float64 
 2   Start_Position                   587201 non-null  float64 
 3   Formula_Avg_Speed_kmh            587201 non-null  float64 
 4   Humidity_%                       587201 non-null  float64 
 5   Champ_Points                     587201 non-null  float64 
 6   Champ_Position                   587201 non-null  float64 
 7   race_yea

XGBoostError: [01:59:48] C:\actions-runner\_work\xgboost\xgboost\src\data\../data/cat_container.h:28: Invalid new DataFrame input for the: 25th feature (0-based). The data type doesn't match the one used in the training dataset. Both should be either numeric or categorical. For a categorical feature, the index type must match between the training and test set.

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import time
import sys

print(f"--- Starting F1 Lap Time Model Training (v6, Category Fix) ---")

# --- 1. Define Features & Target ---
TARGET = 'Lap_Time_Seconds' 

NUMERIC_FEATURES = [
    'Len_Circuit_inkm', 'Laps', 'Start_Position', 'Formula_Avg_Speed_kmh',
    'Humidity_%', 'Champ_Points', 'Champ_Position', 'race_year',
    'seq', 'position', 'points', 'Corners_in_Lap', 'Tire_Degradation_Factor_per_Lap',
    'Pit_Stop_Duration_Seconds', 'Ambient_Temperature_Celsius', 
    'Track_Temperature_Celsius', 'air', 'ground', 'starts', 'finishes',
    'with_points', 'podiums', 'wins'
]

CATEGORICAL_FEATURES = [
    'Rider_ID', 'Formula_category_x', 'Formula_Track_Condition',
    'Tire_Compound', 'Penalty', 'Session', 'Formula_shortname',
    'circuit_name', 'weather', 'track'
]

# This will store the learned categories from the training data
CATEGORY_DTYPES = {}

# --- 2. Load Your Training Data ---
try:
    print("Loading 'train.csv'...")
    df_raw = pd.read_csv('train.csv')
    print(f"Loaded 'train.csv' with {df_raw.shape[0]} rows.")
except FileNotFoundError:
    print("FATAL ERROR: 'train.csv' not found.")
    sys.exit()

# --- 3. Feature Engineering Function ---
def engineer_features(df):
    """Creates new features."""
    print("Engineering features...")
    
    # We must operate on a local copy of the feature lists
    num_features = NUMERIC_FEATURES.copy()
    
    if 'weather' in df.columns:
        df['is_raining'] = df['weather'].astype(str).str.contains('Rain', case=False).fillna(0)
        if 'is_raining' not in num_features:
            num_features.append('is_raining')

    if 'Track_Temperature_Celsius' in df.columns and 'Ambient_Temperature_Celsius' in df.columns:
        track_temp = pd.to_numeric(df['Track_Temperature_Celsius'], errors='coerce')
        ambient_temp = pd.to_numeric(df['Ambient_Temperature_Celsius'], errors='coerce')
        df['temp_diff'] = track_temp - ambient_temp
        
        if 'temp_diff' not in num_features:
            num_features.append('temp_diff')
            
    return df, num_features

# --- 4. Apply Feature Engineering to Training Data ---
df, num_features_list = engineer_features(df_raw.copy())

# --- 5. Data Type Conversion (Training Data) ---
print("Converting data types and learning categories...")
for col in num_features_list:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce') 
        
for col in CATEGORICAL_FEATURES:
    if col in df.columns:
        # Convert to category and store the learned categories
        df[col] = df[col].astype('category')
        CATEGORY_DTYPES[col] = df[col].dtype

# Safety checks
num_features_list = [col for col in num_features_list if col in df.columns]
cat_features_list = [col for col in CATEGORICAL_FEATURES if col in df.columns]

if TARGET not in df.columns:
    print(f"FATAL ERROR: Target column '{TARGET}' not found.")
    sys.exit()

# --- 6. Create Data Splits ---
X = df.drop(TARGET, axis=1)
y = df[TARGET]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training on {len(X_train)} samples, validating on {len(X_val)} samples.")

# --- 7. Set up Preprocessing ---
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features_list)
    ],
    remainder='drop'
)

# --- 8. Apply Preprocessing Manually ---
print("Applying preprocessing...")
preprocessor.fit(X_train[num_features_list])

def preprocess_data(df, processor, num_feats, cat_feats, cat_dtypes):
    """
    Applies scaling to numeric data and enforces correct category
    dtypes on categorical data.
    """
    # 1. Transform the numeric features
    scaled_numeric_data = processor.transform(df[num_feats])
    df_numeric_scaled = pd.DataFrame(scaled_numeric_data, columns=num_feats, index=df.index)
    
    # 2. Get the categorical features and apply *trained* dtypes
    df_categorical = pd.DataFrame(index=df.index)
    for col in cat_feats:
        # Use pd.Categorical to enforce the learned categories
        # Any new, unseen categories in the test set will become NaN
        df_categorical[col] = pd.Categorical(df[col], categories=cat_dtypes[col].categories)
    
    # 3. Concatenate
    df_final = pd.concat([df_numeric_scaled, df_categorical], axis=1)
    return df_final

# Apply to train and validation sets
X_train_final = preprocess_data(X_train, preprocessor, num_features_list, cat_features_list, CATEGORY_DTYPES)
X_val_final = preprocess_data(X_val, preprocessor, num_features_list, cat_features_list, CATEGORY_DTYPES)

print("Preprocessing complete.")

# --- 9. Define the XGBoost Model (CPU Fallback) ---
print("Defining model to run on CPU...")
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    early_stopping_rounds=50,
    tree_method='hist',      # Use the fast CPU method
    enable_categorical=True
)

# --- 10. Train the Model ---
print("\n🏎️  Starting model training...")
start_time = time.time()

model.fit(
    X_train_final, 
    y_train,
    eval_set=[(X_val_final, y_val)], 
    verbose=False 
)

end_time = time.time()
print(f"✅ Training complete in {end_time - start_time:.2f} seconds.")

# --- 11. Evaluate Your New Model ---
y_pred = model.predict(X_val_final)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"\n--- 🏁 Model Evaluation ---")
print(f"Validation RMSE: {rmse:.4f} seconds (Lower is better)")

# --- 12. Load 'test.csv' for Final Predictions ---
print("\nLoading 'test.csv' for final predictions...")
try:
    df_test_raw = pd.read_csv('test.csv')
    submission_ids = df_test_raw['id']
except FileNotFoundError:
    print("FATAL ERROR: 'test.csv' not found.")
    sys.exit()

# --- 13. Apply the SAME Transformations to Test Data ---
print("Applying transformations to test data...")
# 1. Apply feature engineering
df_test_engineered, test_num_feats = engineer_features(df_test_raw.copy())

# 2. Ensure all *numeric* columns are numeric (in case they weren't)
for col in test_num_feats:
    if col in df_test_engineered.columns:
        df_test_engineered[col] = pd.to_numeric(df_test_engineered[col], errors='coerce')
# Note: We do *not* convert categorical columns yet

# 3. Apply the full preprocessing function
# This will scale the numeric columns and *enforce the trained categories*
df_test_final = preprocess_data(
    df_test_engineered, 
    preprocessor, 
    num_features_list,  # Use the *original* list the preprocessor was trained on
    cat_features_list, 
    CATEGORY_DTYPES
)

# --- 14. Make Final Predictions ---
print("Making predictions on test data...")
# This should now work, as the dtypes are identical
final_predictions = model.predict(df_test_final)

# --- 15. Create Submission File ---
print("Creating 'submission.csv'...")
submission_df = pd.DataFrame({
    'id': submission_ids,
    'Lap_Time_Seconds': final_predictions
})
# Handle any potential NaNs in prediction (if test data had unseen categories)
submission_df['Lap_Time_Seconds'] = submission_df['Lap_Time_Seconds'].fillna(y_train.mean())
submission_df['Lap_Time_Seconds'] = submission_df['Lap_Time_Seconds'].apply(lambda x: max(0, x))
submission_df.to_csv('submission.csv', index=False)

print("\n✅ Done! Your 'submission.csv' file is ready to be uploaded.")

--- Starting F1 Lap Time Model Training (v6, Category Fix) ---
Loading 'train.csv'...
Loaded 'train.csv' with 734002 rows.
Engineering features...
Converting data types and learning categories...
Training on 587201 samples, validating on 146801 samples.
Applying preprocessing...
Preprocessing complete.
Defining model to run on CPU...

🏎️  Starting model training...
✅ Training complete in 72.89 seconds.

--- 🏁 Model Evaluation ---
Validation RMSE: 11.2226 seconds (Lower is better)

Loading 'test.csv' for final predictions...
Applying transformations to test data...
Engineering features...
Making predictions on test data...
Creating 'submission.csv'...

✅ Done! Your 'submission.csv' file is ready to be uploaded.


In [3]:
# --- 1. Load Data ---
df = pd.read_csv('train.csv')

# --- 2. Apply CORRECT Filters ---
print(f"Original data: {len(df)} rows")

# --- Filter 1: Session ---
# --- YOU MUST EDIT 'Race' ---
# Change 'Race' to whatever you found in your investigation
RACE_SESSION_NAME = 'Race' 
df_clean = df[df['Session'] == RACE_SESSION_NAME].copy()
print(f"After 'Session' filter: {len(df_clean)} rows")

# --- Filter 2: First Lap ---
# We use 'seq' (current lap), not 'Laps' (total laps)
df_clean = df_clean[df_clean['seq'] > 1].copy()
print(f"After 'First Lap' filter: {len(df_clean)} rows")

# --- Filter 3: Pit Stops ---
df_clean = df_clean[df_clean['Pit_Stop_Duration_Seconds'].fillna(0) == 0].copy()
print(f"After 'Pit Stop' filter: {len(df_clean)} rows")

# --- Filter 4: Penalties / DNFs ---
# A much better filter: keep all laps *except* DNF/DNS
interrupted_laps = ['DNF', 'DNS']
df_clean = df_clean[~df_clean['Penalty'].isin(interrupted_laps)].copy()
print(f"After 'Penalty' filter: {len(df_clean)} rows")

# --- Filter 5: Track Conditions ---
# This is probably correct, but check your investigation output
valid_conditions = ['Dry', 'Wet']
df_clean = df_clean[df_clean['Formula_Track_Condition'].isin(valid_conditions)].copy()
print(f"After 'Track Condition' filter: {len(df_clean)} rows")

# --- Final Check ---
print(f"Final cleaned data: {len(df_clean)} rows")

# Now, use this 'df_clean' for your feature engineering and training

Original data: 734002 rows
After 'Session' filter: 105815 rows
After 'First Lap' filter: 98599 rows
After 'Pit Stop' filter: 0 rows
After 'Penalty' filter: 0 rows
After 'Track Condition' filter: 0 rows
Final cleaned data: 0 rows


In [4]:
import pandas as pd
import numpy as np

# Load your full training data
try:
    df = pd.read_csv('train.csv')
except FileNotFoundError:
    print("FATAL ERROR: 'train.csv' not found.")
    exit()

print("--- Investigating Columns for New Filters ---")

# --- 1. Investigate 'Lap_Time_Seconds' (to find a good cutoff) ---
print("\n[ Lap_Time_Seconds Stats ]")
# We'll look at the "quantiles" (percentiles)
# A 'normal' lap will be near the 50% (median)
# A 'pit stop' lap will be up near the 90-95%
print(df['Lap_Time_Seconds'].describe(percentiles=[.25, .5, .75, .90, .95, .99]))
print("--------------------")


# --- 2. Investigate 'Pit_Stop_Duration_Seconds' (to confirm it's bad) ---
print("\n[ Pit_Stop_Duration_Seconds Stats ]")
# This will show us what the values *actually* look like
print(df['Pit_Stop_Duration_Seconds'].describe())
print("\n[ Example values for Pit_Stop_Duration ]")
print(df['Pit_Stop_Duration_Seconds'].value_counts().head(10))
print("--------------------")


# --- 3. Investigate 'Penalty' ---
print("\n[ Penalty Values ]")
print(df['Penalty'].value_counts(dropna=False))
print("--------------------")


# --- 4. Investigate 'Formula_Track_Condition' ---
print("\n[ Track Condition Values ]")
print(df['Formula_Track_Condition'].value_counts(dropna=False))
print("--------------------")

--- Investigating Columns for New Filters ---

[ Lap_Time_Seconds Stats ]
count    734002.000000
mean         89.996788
std          11.531972
min          70.001000
25%          79.989000
50%          89.970000
75%          99.914000
90%         106.053000
95%         108.028000
99%         109.599000
max         109.999000
Name: Lap_Time_Seconds, dtype: float64
--------------------

[ Pit_Stop_Duration_Seconds Stats ]
count    734002.000000
mean          3.502278
std           0.868022
min           2.000000
25%           2.750000
50%           3.510000
75%           4.250000
max           5.000000
Name: Pit_Stop_Duration_Seconds, dtype: float64

[ Example values for Pit_Stop_Duration ]
Pit_Stop_Duration_Seconds
2.69    2963
3.83    2898
3.88    2882
2.46    2860
3.79    2855
4.97    2840
3.90    2839
3.51    2838
2.04    2831
2.82    2818
Name: count, dtype: int64
--------------------

[ Penalty Values ]
Penalty
DNS             123474
NaN             122881
+3s             122875
DN

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import time
import sys

print(f"--- Starting F1 Lap Time Model Training (v8, Corrected Filters) ---")

# --- 1. Define Features & Target ---
TARGET = 'Lap_Time_Seconds' 

# Base features (we will add more)
NUMERIC_FEATURES = [
    'Len_Circuit_inkm', 'Laps', 'Start_Position', 'Formula_Avg_Speed_kmh',
    'Humidity_%', 'Champ_Points', 'Champ_Position', 'race_year',
    'seq', 'position', 'points', 'Corners_in_Lap', 'Tire_Degradation_Factor_per_Lap',
    # 'Pit_Stop_Duration_Seconds' is now a real feature, not a filter!
    'Pit_Stop_Duration_Seconds', 
    'Ambient_Temperature_Celsius', 
    'Track_Temperature_Celsius', 'air', 'ground', 'starts', 'finishes',
    'with_points', 'podiums', 'wins'
]
CATEGORICAL_FEATURES = [
    'Rider_ID', 'Formula_category_x', 'Formula_Track_Condition',
    'Tire_Compound', 'Penalty', 'Session', 'Formula_shortname',
    'circuit_name', 'weather', 'track'
]
# This will store the learned categories from the training data
CATEGORY_DTYPES = {}

# --- 2. Load Your Training Data ---
try:
    print("Loading 'train.csv'...")
    df_raw = pd.read_csv('train.csv')
    print(f"Loaded 'train.csv' with {df_raw.shape[0]} rows.")
except FileNotFoundError:
    print("FATAL ERROR: 'train.csv' not found.")
    sys.exit()

# --- 3. Apply CORRECTED Data Filters ---
print(f"Original data: {len(df_raw)} rows")

# Filter 1: Session
df_clean = df_raw[df_raw['Session'] == 'Race'].copy()
print(f"After 'Session' filter: {len(df_clean)} rows")

# Filter 2: First Lap (using 'seq' as current lap)
df_clean = df_clean[df_clean['seq'] > 1].copy()
print(f"After 'First Lap' filter: {len(df_clean)} rows")

# Filter 3: Penalties (Remove non-racing laps)
# Keep NaN, +3s, +5s. Remove DNF, DNS, Ride Through.
bad_laps = ['DNF', 'DNS', 'Ride Through']
df_clean = df_clean[~df_clean['Penalty'].isin(bad_laps)].copy()
print(f"After 'Penalty' filter: {len(df_clean)} rows")

# NOTE: We no longer filter on 'Pit_Stop_Duration_Seconds'
# NOTE: We no longer need to filter on 'Formula_Track_Condition'

if len(df_clean) == 0:
    print("FATAL ERROR: Filtering resulted in 0 rows. Check filters again.")
    sys.exit()

# --- 4. ADVANCED Feature Engineering ---
def engineer_features(df, num_features):
    """Creates new, high-value features."""
    print("Engineering advanced features...")
    
    num_features_copy = num_features.copy()
    
    # Feature 1: Tire Age
    # This is still a guess, but a much better one.
    # We create a 'stint_id' by looking for 'Penalty' == 'Ride Through',
    # as that's the only other way to know a "slow" lap.
    # A better way would be to see if Tire_Compound changes.
    df['is_ride_through'] = (df['Penalty'] == 'Ride Through').astype(int)
    df['stint_id'] = df.groupby('Rider_ID')['is_ride_through'].cumsum()
    df['tire_age'] = df.groupby(['Rider_ID', 'stint_id']).cumcount()
    if 'tire_age' not in num_features_copy:
        num_features_copy.append('tire_age')
    print("Engineered 'tire_age' feature.")

    # Feature 2: Weather Interactions
    if 'weather' in df.columns:
        df['is_raining'] = df['weather'].astype(str).str.contains('Rain', case=False).fillna(0)
        if 'is_raining' not in num_features_copy:
            num_features_copy.append('is_raining')

    if 'Track_Temperature_Celsius' in df.columns:
        track_temp = pd.to_numeric(df['Track_Temperature_Celsius'], errors='coerce')
        ambient_temp = pd.to_numeric(df['Ambient_Temperature_Celsius'], errors='coerce')
        df['temp_diff'] = track_temp - ambient_temp
        if 'temp_diff' not in num_features_copy:
            num_features_copy.append('temp_diff')
    
    # Feature 3: Driver/Team Recent Form
    df = df.sort_values(by=['race_year', 'seq'])
    if 'points' in df.columns:
        df['driver_rolling_points'] = df.groupby('Rider_ID')['points'].rolling(window=5, min_periods=1).mean().reset_index(level=0, drop=True)
        if 'driver_rolling_points' not in num_features_copy:
            num_features_copy.append('driver_rolling_points')
        print("Engineered 'driver_rolling_points' feature.")

    return df, num_features_copy

# Apply FE *only* to the clean data
df_featured, num_features_list = engineer_features(df_clean.copy(), NUMERIC_FEATURES)

# --- 5. Data Type Conversion (Training Data) ---
print("Converting data types and learning categories...")
for col in num_features_list:
    if col in df_featured.columns:
        df_featured[col] = pd.to_numeric(df_featured[col], errors='coerce') 
        
for col in CATEGORICAL_FEATURES:
    if col in df_featured.columns:
        df_featured[col] = df_featured[col].astype('category')
        CATEGORY_DTYPES[col] = df_featured[col].dtype

cat_features_list = [col for col in CATEGORICAL_FEATURES if col in df_featured.columns]

if TARGET not in df_featured.columns:
    print(f"FATAL ERROR: Target column '{TARGET}' not found.")
    sys.exit()

# --- 6. Create Data Splits ---
X = df_featured.drop(TARGET, axis=1)
y = df_featured[TARGET]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training on {len(X_train)} samples, validating on {len(X_val)} samples.")

# --- 7. Set up Preprocessing ---
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features_list)
    ],
    remainder='drop'
)

# --- 8. Apply Preprocessing Manually ---
print("Applying preprocessing...")
preprocessor.fit(X_train[num_features_list])

def preprocess_data(df, processor, num_feats, cat_feats, cat_dtypes):
    """Applies scaling and enforces category dtypes."""
    df_numeric_scaled = pd.DataFrame(processor.transform(df[num_feats]), columns=num_feats, index=df.index)
    
    df_categorical = pd.DataFrame(index=df.index)
    for col in cat_feats:
        df_categorical[col] = pd.Categorical(df[col], categories=cat_dtypes[col].categories)
    
    df_final = pd.concat([df_numeric_scaled, df_categorical], axis=1)
    return df_final

X_train_final = preprocess_data(X_train, preprocessor, num_features_list, cat_features_list, CATEGORY_DTYPES)
X_val_final = preprocess_data(X_val, preprocessor, num_features_list, cat_features_list, CATEGORY_DTYPES)
print("Preprocessing complete.")

# --- 9. Define the XGBoost Model (Tuned Parameters) ---
print("Defining tuned model to run on CPU...")
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=2000,
    learning_rate=0.02,
    max_depth=8,
    subsample=0.7,
    colsample_bytree=0.7,
    random_state=42,
    early_stopping_rounds=100,
    tree_method='hist',
    enable_categorical=True
)

# --- 10. Train the Model ---
print("\n🏎️  Starting model training...")
start_time = time.time()
model.fit(
    X_train_final, 
    y_train,
    eval_set=[(X_val_final, y_val)], 
    verbose=False 
)
end_time = time.time()
print(f"✅ Training complete in {end_time - start_time:.2f} seconds.")

# --- 11. Evaluate Your New Model ---
y_pred = model.predict(X_val_final)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"\n--- 🏁 Model Evaluation ---")
print(f"New Validation RMSE: {rmse:.4f} seconds (This should be much better)")

# --- 12. Load 'test.csv' for Final Predictions ---
print("\nLoading 'test.csv' for final predictions...")
try:
    df_test_raw = pd.read_csv('test.csv')
    submission_ids = df_test_raw['id']
except FileNotFoundError:
    print("FATAL ERROR: 'test.csv' not found.")
    sys.exit()

# --- 13. Apply the SAME Transformations to Test Data ---
print("Applying transformations to test data...")
# 1. Apply feature engineering
#    NOTE: We do NOT filter the test set.
df_test_engineered, test_num_feats = engineer_features(df_test_raw.copy(), NUMERIC_FEATURES)

# 2. Ensure all *numeric* columns are numeric
for col in test_num_feats:
    if col in df_test_engineered.columns:
        df_test_engineered[col] = pd.to_numeric(df_test_engineered[col], errors='coerce')

# 3. Apply the full preprocessing function
df_test_final = preprocess_data(
    df_test_engineered, 
    preprocessor, 
    num_features_list,
    cat_features_list, 
    CATEGORY_DTYPES
)

# --- 14. Make Final Predictions ---
print("Making predictions on test data...")
final_predictions = model.predict(df_test_final)

# --- 15. Create Submission File ---
print("Creating 'submission.csv'...")
submission_df = pd.DataFrame({
    'id': submission_ids,
    'Lap_Time_Seconds': final_predictions
})
# Fill any rows that became NaN (due to unseen categories) with the average lap time
submission_df['Lap_Time_Seconds'] = submission_df['Lap_Time_Seconds'].fillna(y_train.mean())
submission_df['Lap_Time_Seconds'] = submission_df['Lap_Time_Seconds'].apply(lambda x: max(0, x))
submission_df.to_csv('submission.csv', index=False)

print("\n✅ Done! Your new 'submission.csv' is ready. Submit this and check your score.")

In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import time
import sys

print(f"--- Starting F1 Lap Time Model Training (v8, Corrected Filters) ---")

# --- 1. Define Features & Target ---
TARGET = 'Lap_Time_Seconds' 
NUMERIC_FEATURES = [
    'Len_Circuit_inkm', 'Laps', 'Start_Position', 'Formula_Avg_Speed_kmh',
    'Humidity_%', 'Champ_Points', 'Champ_Position', 'race_year',
    'seq', 'position', 'points', 'Corners_in_Lap', 'Tire_Degradation_Factor_per_Lap',
    'Pit_Stop_Duration_Seconds', 'Ambient_Temperature_Celsius', 
    'Track_Temperature_Celsius', 'air', 'ground', 'starts', 'finishes',
    'with_points', 'podiums', 'wins'
]
CATEGORICAL_FEATURES = [
    'Rider_ID', 'Formula_category_x', 'Formula_Track_Condition',
    'Tire_Compound', 'Penalty', 'Session', 'Formula_shortname',
    'circuit_name', 'weather', 'track'
]
CATEGORY_DTYPES = {}

# --- 2. Load Your Training Data ---
try:
    print("Loading 'train.csv'...")
    df_raw = pd.read_csv('train.csv')
    print(f"Loaded 'train.csv' with {df_raw.shape[0]} rows.")
except FileNotFoundError:
    print("FATAL ERROR: 'train.csv' not found.")
    sys.exit()

# --- 3. Apply CORRECTED Data Filters ---
print(f"Original data: {len(df_raw)} rows")
df_clean = df_raw[df_raw['Session'] == 'Race'].copy()
print(f"After 'Session' filter: {len(df_clean)} rows")
df_clean = df_clean[df_clean['seq'] > 1].copy()
print(f"After 'First Lap' filter: {len(df_clean)} rows")
bad_laps = ['DNF', 'DNS', 'Ride Through']
df_clean = df_clean[~df_clean['Penalty'].isin(bad_laps)].copy()
print(f"After 'Penalty' filter: {len(df_clean)} rows")
if len(df_clean) == 0:
    print("FATAL ERROR: Filtering resulted in 0 rows. Check filters again.")
    sys.exit()

# --- 4. ADVANCED Feature Engineering ---
def engineer_features(df, num_features):
    print("Engineering advanced features...")
    num_features_copy = num_features.copy()
    
    df['is_ride_through'] = (df['Penalty'] == 'Ride Through').astype(int)
    df['stint_id'] = df.groupby('Rider_ID')['is_ride_through'].cumsum()
    df['tire_age'] = df.groupby(['Rider_ID', 'stint_id']).cumcount()
    if 'tire_age' not in num_features_copy:
        num_features_copy.append('tire_age')

    if 'weather' in df.columns:
        df['is_raining'] = df['weather'].astype(str).str.contains('Rain', case=False).fillna(0)
        if 'is_raining' not in num_features_copy:
            num_features_copy.append('is_raining')

    if 'Track_Temperature_Celsius' in df.columns:
        track_temp = pd.to_numeric(df['Track_Temperature_Celsius'], errors='coerce')
        ambient_temp = pd.to_numeric(df['Ambient_Temperature_Celsius'], errors='coerce')
        df['temp_diff'] = track_temp - ambient_temp
        if 'temp_diff' not in num_features_copy:
            num_features_copy.append('temp_diff')
    
    df = df.sort_values(by=['race_year', 'seq'])
    if 'points' in df.columns:
        df['driver_rolling_points'] = df.groupby('Rider_ID')['points'].rolling(window=5, min_periods=1).mean().reset_index(level=0, drop=True)
        if 'driver_rolling_points' not in num_features_copy:
            num_features_copy.append('driver_rolling_points')

    return df, num_features_copy

df_featured, num_features_list = engineer_features(df_clean.copy(), NUMERIC_FEATURES)

# --- 5. Data Type Conversion (Training Data) ---
print("Converting data types and learning categories...")
for col in num_features_list:
    if col in df_featured.columns:
        df_featured[col] = pd.to_numeric(df_featured[col], errors='coerce') 
for col in CATEGORICAL_FEATURES:
    if col in df_featured.columns:
        df_featured[col] = df_featured[col].astype('category')
        CATEGORY_DTYPES[col] = df_featured[col].dtype
cat_features_list = [col for col in CATEGORICAL_FEATURES if col in df_featured.columns]
if TARGET not in df_featured.columns:
    print("FATAL ERROR: Target column not found.")
    sys.exit()

# --- 6. Create Data Splits ---
X = df_featured.drop(TARGET, axis=1)
y = df_featured[TARGET]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training on {len(X_train)} samples, validating on {len(X_val)} samples.")

# --- 7. Set up Preprocessing ---
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer, num_features_list)],
    remainder='drop'
)

# --- 8. Apply Preprocessing Manually ---
print("Applying preprocessing...")
preprocessor.fit(X_train[num_features_list])
def preprocess_data(df, processor, num_feats, cat_feats, cat_dtypes):
    df_numeric_scaled = pd.DataFrame(processor.transform(df[num_feats]), columns=num_feats, index=df.index)
    df_categorical = pd.DataFrame(index=df.index)
    for col in cat_feats:
        df_categorical[col] = pd.Categorical(df[col], categories=cat_dtypes[col].categories)
    df_final = pd.concat([df_numeric_scaled, df_categorical], axis=1)
    return df_final
X_train_final = preprocess_data(X_train, preprocessor, num_features_list, cat_features_list, CATEGORY_DTYPES)
X_val_final = preprocess_data(X_val, preprocessor, num_features_list, cat_features_list, CATEGORY_DTYPES)
print("Preprocessing complete.")

# --- 9. Define the XGBoost Model (TUNING SECTION) ---
print("Defining tuned model to run on CPU...")
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    
    # --- KNOB 1: `n_estimators` & `early_stopping_rounds` ---
    # We set n_estimators high and let early_stopping find the best number.
    # This is already tuned for you!
    n_estimators=3000,           # Set this high
    early_stopping_rounds=100,   # Stops when the score hasn't improved for 100 trees
    
    # --- KNOB 2: `learning_rate` ---
    # Smaller is often more accurate but slower.
    # Try 0.05, then 0.02, then 0.01
    learning_rate=0.02,
    
    # --- KNOB 3: `max_depth` ---
    # Deeper trees are more complex. Risk of overfitting.
    # Try 6, then 8, then 10.
    max_depth=8,
    
    # --- KNOB 4: Regularization ---
    # Helps prevent overfitting. Values from 0.6 to 0.9 are good.
    subsample=0.7,
    colsample_bytree=0.7,
    
    random_state=42,
    tree_method='hist',
    enable_categorical=True
)

# --- 10. Train the Model ---
print("\n🏎️  Starting model training...")
start_time = time.time()
model.fit(
    X_train_final, 
    y_train,
    eval_set=[(X_val_final, y_val)], 
    verbose=False 
)
end_time = time.time()
print(f"✅ Training complete in {end_time - start_time:.2f} seconds.")

# --- 11. Evaluate Your New Model ---
y_pred = model.predict(X_val_final)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"\n--- 🏁 Model Evaluation ---")
print(f"New Validation RMSE: {rmse:.4f} seconds (Write this down!)")

# --- 12. Load 'test.csv' for Final Predictions ---
print("\nLoading 'test.csv' for final predictions...")
try:
    df_test_raw = pd.read_csv('test.csv')
    submission_ids = df_test_raw['id']
except FileNotFoundError:
    print("FATAL ERROR: 'test.csv' not found.")
    sys.exit()

# --- 13. Apply the SAME Transformations to Test Data ---
print("Applying transformations to test data...")
df_test_engineered, test_num_feats = engineer_features(df_test_raw.copy(), NUMERIC_FEATURES)
for col in test_num_feats:
    if col in df_test_engineered.columns:
        df_test_engineered[col] = pd.to_numeric(df_test_engineered[col], errors='coerce')
df_test_final = preprocess_data(
    df_test_engineered, 
    preprocessor, 
    num_features_list,
    cat_features_list, 
    CATEGORY_DTYPES
)

# --- 14. Make Final Predictions ---
print("Making predictions on test data...")
final_predictions = model.predict(df_test_final)

# --- 15. Create Submission File ---
print("Creating 'submission.csv'...")
submission_df = pd.DataFrame({
    'id': submission_ids,
    'Lap_Time_Seconds': final_predictions
})
submission_df['Lap_Time_Seconds'] = submission_df['Lap_Time_Seconds'].fillna(y_train.mean())
submission_df['Lap_Time_Seconds'] = submission_df['Lap_Time_Seconds'].apply(lambda x: max(0, x))
submission_df.to_csv('submission1.csv', index=False)

print("\n✅ Done! Your new 'submission.csv' is ready.")

--- Starting F1 Lap Time Model Training (v8, Corrected Filters) ---
Loading 'train.csv'...
Loaded 'train.csv' with 734002 rows.
Original data: 734002 rows
After 'Session' filter: 105815 rows
After 'First Lap' filter: 98599 rows
After 'Penalty' filter: 49074 rows
Engineering advanced features...
Converting data types and learning categories...
Training on 39259 samples, validating on 9815 samples.
Applying preprocessing...
Preprocessing complete.
Defining tuned model to run on CPU...

🏎️  Starting model training...
✅ Training complete in 1.88 seconds.

--- 🏁 Model Evaluation ---
New Validation RMSE: 11.5441 seconds (Write this down!)

Loading 'test.csv' for final predictions...
Applying transformations to test data...
Engineering advanced features...
Making predictions on test data...
Creating 'submission.csv'...

✅ Done! Your new 'submission.csv' is ready.


In [1]:
!pip install pandas scikit-learn xgboost

  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ------------------- -------------------- 5.5/11.3 MB 25.7 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.3 MB 26.1 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 22.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   ---------------------- ----------------- 5.0/8.9 MB 25.2 MB/s eta 0:00:01
   --------------------------------- ------ 7.3/8.9 MB 18.9 MB/s eta 0:00:01
   ---------------------------------------- 8.9/8.9 MB 15.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   - -------------------------------------- 2.9/72.0 MB 13.9 MB/s eta 0:00:05
   --- ------------------------------------ 5.8/72.0 MB 13.6 MB/s eta 0:00:05
   ---- ----------------------------


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\manna\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import time
import sys
from scipy.stats import uniform, randint

print(f"--- Starting F1 Lap Time Model Training (v9, Auto-Tuning) ---")

# --- 1. Define Features & Target ---
TARGET = 'Lap_Time_Seconds' 
NUMERIC_FEATURES = [
    'Len_Circuit_inkm', 'Laps', 'Start_Position', 'Formula_Avg_Speed_kmh',
    'Humidity_%', 'Champ_Points', 'Champ_Position', 'race_year',
    'seq', 'position', 'points', 'Corners_in_Lap', 'Tire_Degradation_Factor_per_Lap',
    'Pit_Stop_Duration_Seconds', 'Ambient_Temperature_Celsius', 
    'Track_Temperature_Celsius', 'air', 'ground', 'starts', 'finishes',
    'with_points', 'podiums', 'wins'
]
CATEGORICAL_FEATURES = [
    'Rider_ID', 'Formula_category_x', 'Formula_Track_Condition',
    'Tire_Compound', 'Penalty', 'Session', 'Formula_shortname',
    'circuit_name', 'weather', 'track'
]
CATEGORY_DTYPES = {}

# --- 2. Load Your Training Data ---
try:
    print("Loading 'train.csv'...")
    df_raw = pd.read_csv('train.csv')
    print(f"Loaded 'train.csv' with {df_raw.shape[0]} rows.")
except FileNotFoundError:
    print("FATAL ERROR: 'train.csv' not found.")
    sys.exit()

# --- 3. Apply CORRECTED Data Filters ---
print(f"Original data: {len(df_raw)} rows")
df_clean = df_raw[df_raw['Session'] == 'Race'].copy()
df_clean = df_clean[df_clean['seq'] > 1].copy()
bad_laps = ['DNF', 'DNS', 'Ride Through']
df_clean = df_clean[~df_clean['Penalty'].isin(bad_laps)].copy()
print(f"After all filters, {len(df_clean)} rows remain for training.")
if len(df_clean) == 0:
    print("FATAL ERROR: Filtering resulted in 0 rows.")
    sys.exit()

# --- 4. ADVANCED Feature Engineering (v2) ---
def engineer_features(df, num_features):
    """Creates new, high-value features."""
    print("Engineering advanced features (v2)...")
    
    num_features_copy = num_features.copy()
    
    # --- Feature 1: Tire Age (Improved) ---
    # A 'stint' is defined by a driver AND a tire compound.
    # We sort by lap ('seq') to ensure the order is correct.
    df = df.sort_values(by=['race_year', 'Rider_ID', 'seq'])
    
    # Create a 'stint_id' that changes when the Tire_Compound changes
    df['stint_id'] = (df['Tire_Compound'] != df.groupby('Rider_ID')['Tire_Compound'].shift()).astype(int).cumsum()
    
    # Now, 'tire_age' is the lap count *within that stint*
    df['tire_age'] = df.groupby(['Rider_ID', 'stint_id']).cumcount()
    if 'tire_age' not in num_features_copy:
        num_features_copy.append('tire_age')
    print("Engineered improved 'tire_age' feature.")

    # --- Feature 2: Weather Interactions ---
    if 'weather' in df.columns:
        df['is_raining'] = df['weather'].astype(str).str.contains('Rain', case=False).fillna(0)
        if 'is_raining' not in num_features_copy:
            num_features_copy.append('is_raining')

    if 'Track_Temperature_Celsius' in df.columns:
        track_temp = pd.to_numeric(df['Track_Temperature_Celsius'], errors='coerce')
        ambient_temp = pd.to_numeric(df['Ambient_Temperature_Celsius'], errors='coerce')
        df['temp_diff'] = track_temp - ambient_temp
        if 'temp_diff' not in num_features_copy:
            num_features_copy.append('temp_diff')

        # --- New Interaction Feature ---
        df['tire_age_x_track_temp'] = df['tire_age'] * track_temp
        if 'tire_age_x_track_temp' not in num_features_copy:
            num_features_copy.append('tire_age_x_track_temp')
            
    # --- Feature 3: Driver/Team Recent Form ---
    if 'points' in df.columns:
        df['driver_rolling_points'] = df.groupby('Rider_ID')['points'].rolling(window=5, min_periods=1).mean().reset_index(level=0, drop=True)
        if 'driver_rolling_points' not in num_features_copy:
            num_features_copy.append('driver_rolling_points')

    return df, num_features_copy

df_featured, num_features_list = engineer_features(df_clean.copy(), NUMERIC_FEATURES)

# --- 5. Data Type Conversion (Training Data) ---
print("Converting data types and learning categories...")
for col in num_features_list:
    if col in df_featured.columns:
        df_featured[col] = pd.to_numeric(df_featured[col], errors='coerce') 
for col in CATEGORICAL_FEATURES:
    if col in df_featured.columns:
        df_featured[col] = df_featured[col].astype('category')
        CATEGORY_DTYPES[col] = df_featured[col].dtype
cat_features_list = [col for col in CATEGORICAL_FEATURES if col in df_featured.columns]
if TARGET not in df_featured.columns:
    print("FATAL ERROR: Target column not found.")
    sys.exit()

# --- 6. Create Data Splits ---
X = df_featured.drop(TARGET, axis=1)
y = df_featured[TARGET]
# We create a final hold-out test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# And a smaller validation set for the tuner
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(f"Total training samples: {len(X_train)}")
print(f"Using {len(X_train_sub)} for tuning, {len(X_val)} for validation.")

# --- 7. Set up Preprocessing ---
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer, num_features_list)],
    remainder='drop'
)

# --- 8. Apply Preprocessing Manually ---
print("Applying preprocessing...")
preprocessor.fit(X_train[num_features_list])
def preprocess_data(df, processor, num_feats, cat_feats, cat_dtypes):
    df_numeric_scaled = pd.DataFrame(processor.transform(df[num_feats]), columns=num_feats, index=df.index)
    df_categorical = pd.DataFrame(index=df.index)
    for col in cat_feats:
        df_categorical[col] = pd.Categorical(df[col], categories=cat_dtypes[col].categories)
    df_final = pd.concat([df_numeric_scaled, df_categorical], axis=1)
    return df_final

X_train_final = preprocess_data(X_train, preprocessor, num_features_list, cat_features_list, CATEGORY_DTYPES)
X_test_final = preprocess_data(X_test, preprocessor, num_features_list, cat_features_list, CATEGORY_DTYPES)

# Preprocess the tuning sets
X_train_sub_final = preprocess_data(X_train_sub, preprocessor, num_features_list, cat_features_list, CATEGORY_DTYPES)
X_val_final = preprocess_data(X_val, preprocessor, num_features_list, cat_features_list, CATEGORY_DTYPES)

print("Preprocessing complete.")

# --- 9. Define the XGBoost Model and Tuning Parameters ---
print("Defining model and tuning search space...")
# Base model
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=2000,           # Start high, will use early stopping
    
    # --- FIX IS HERE ---
    # Moved early_stopping_rounds from .fit() to the constructor
    early_stopping_rounds=100,
    # -------------------
    
    random_state=42,
    tree_method='hist',
    enable_categorical=True
)

# Parameter grid for RandomizedSearchCV
# This will test random combinations of these settings
param_dist = {
    'learning_rate': uniform(0.01, 0.1),  # Try values between 0.01 and 0.11
    'max_depth': randint(6, 12),         # Try depths from 6 to 11
    'subsample': uniform(0.6, 0.4),      # Try 0.6 to 1.0
    'colsample_bytree': uniform(0.6, 0.4) # Try 0.6 to 1.0
}

# Set up the Randomized Search
print("Starting RandomizedSearchCV... This will be slow!")
start_time = time.time()

random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=10,  # Try 10 combinations. Increase to 20 or 30 for better results.
    cv=3,       # 3-fold cross-validation
    scoring='neg_root_mean_squared_error', # The metric to optimize
    n_jobs=-1,  # Use all CPU cores
    verbose=1,  # Show progress
    random_state=42
)

# Fit the search on the *smaller* training subset
# --- FIX IS HERE ---
# We removed early_stopping_rounds from this call
random_search.fit(
    X=X_train_sub_final, 
    y=y_train_sub,
    eval_set=[(X_val_final, y_val)], # This eval_set is for the tuner
    verbose=False
)
# -------------------

end_time = time.time()
print(f"✅ Tuner finished in {(end_time - start_time) / 60:.2f} minutes.")
print(f"Best parameters found: {random_search.best_params_}")
print(f"Best tuning score (RMSE): {-random_search.best_score_:.4f}")

# --- 10. Train the FINAL Model ---
print("\nTraining the final, best model on ALL training data...")
# Get the best model from the search
best_model = random_search.best_estimator_

# Now, retrain it on the *entire* training set

# --- FIX IS HERE ---
# We remove 'early_stopping_rounds' because the 'best_model'
# already has this setting from when it was created in Section 9.
best_model.fit(
    X_train_final, 
    y_train,
    eval_set=[(X_test_final, y_test)], # Use the hold-out test set for stopping
    verbose=False
)
# -------------------

# --- 11. Evaluate Your New Model on the Test Set ---
y_pred = best_model.predict(X_test_final)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"\n--- 🏁 FINAL Model Evaluation ---")
print(f"Final Test Set RMSE: {rmse:.4f} seconds")
print("This is the most reliable score for your new model.")

# ... (The rest of your script, from Section 12 onwards, is fine) ...

# --- 12. Load 'test.csv' for Final Predictions ---
print("\nLoading 'test.csv' for final predictions...")
df_test_raw = pd.read_csv('test.csv')
submission_ids = df_test_raw['id']

# --- 13. Apply the SAME Transformations to Test Data ---
print("Applying transformations to test data...")
df_test_engineered, test_num_feats = engineer_features(df_test_raw.copy(), NUMERIC_FEATURES)
for col in test_num_feats:
    if col in df_test_engineered.columns:
        df_test_engineered[col] = pd.to_numeric(df_test_engineered[col], errors='coerce')
df_test_final = preprocess_data(
    df_test_engineered, 
    preprocessor, 
    num_features_list,
    cat_features_list, 
    CATEGORY_DTYPES
)

# --- 14. Make Final Predictions ---
print("Making predictions on test data...")
final_predictions = best_model.predict(df_test_final)

# --- 15. Create Submission File ---
print("Creating 'final_tuned_submission.csv'...")
submission_df = pd.DataFrame({
    'id': submission_ids,
    'Lap_Time_Seconds': final_predictions
})
submission_df['Lap_Time_Seconds'] = submission_df['Lap_Time_Seconds'].fillna(y_train.mean())
submission_df['Lap_Time_Seconds'] = submission_df['Lap_Time_Seconds'].apply(lambda x: max(0, x))
submission_df.to_csv('final_tuned_submission.csv', index=False)

print("\n✅ Done! Your new 'final_tuned_submission.csv' is ready.")

--- Starting F1 Lap Time Model Training (v9, Auto-Tuning) ---
Loading 'train.csv'...
Loaded 'train.csv' with 734002 rows.
Original data: 734002 rows
After all filters, 49074 rows remain for training.
Engineering advanced features (v2)...
Engineered improved 'tire_age' feature.
Converting data types and learning categories...
Total training samples: 39259
Using 31407 for tuning, 7852 for validation.
Applying preprocessing...
Preprocessing complete.
Defining model and tuning search space...
Starting RandomizedSearchCV... This will be slow!
Fitting 3 folds for each of 10 candidates, totalling 30 fits
✅ Tuner finished in 0.62 minutes.
Best parameters found: {'colsample_bytree': np.float64(0.7334834444556088), 'learning_rate': np.float64(0.02428668179219408), 'max_depth': 8, 'subsample': np.float64(0.608233797718321)}
Best tuning score (RMSE): 11.4910

Training the final, best model on ALL training data...

--- 🏁 FINAL Model Evaluation ---
Final Test Set RMSE: 11.5207 seconds
This is the mo

In [5]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import time
import sys
from scipy.stats import uniform, randint

print(f"--- Starting F1 Lap Time Model Training (v12, KeyError Fix) ---")

# --- 1. Define Features & Target ---
TARGET = 'Lap_Time_Seconds' 
NUMERIC_FEATURES = [
    'Len_Circuit_inkm', 'Laps', 'Start_Position', 'Formula_Avg_Speed_kmh',
    'Humidity_%', 'Champ_Points', 'Champ_Position', 'race_year',
    'seq', 'position', 'points', 'Corners_in_Lap', 'Tire_Degradation_Factor_per_Lap',
    'Pit_Stop_Duration_Seconds', 'Ambient_Temperature_Celsius', 
    'Track_Temperature_Celsius', 'air', 'ground', 'starts', 'finishes',
    'with_points', 'podiums', 'wins'
]
CATEGORICAL_FEATURES = [
    'Rider_ID', 'Formula_category_x', 'Formula_Track_Condition',
    'Tire_Compound', 'Penalty', 'Session', 'Formula_shortname',
    'circuit_name', 'weather', 'track'
]
CATEGORY_DTYPES = {}

# --- 2. Load Your Training Data ---
try:
    print("Loading 'train.csv'...")
    df_raw = pd.read_csv('train.csv')
    print(f"Loaded 'train.csv' with {df_raw.shape[0]} rows.")
except FileNotFoundError:
    print("FATAL ERROR: 'train.csv' not found.")
    sys.exit()

# --- 3. Apply CORRECTED Data Filters ---
print(f"Original data: {len(df_raw)} rows")
df_clean = df_raw[df_raw['Session'] == 'Race'].copy()
df_clean = df_clean[df_clean['seq'] > 1].copy()
bad_laps = ['DNF', 'DNS', 'Ride Through']
df_clean = df_clean[~df_clean['Penalty'].isin(bad_laps)].copy()
print(f"After all filters, {len(df_clean)} rows remain for training.")
if len(df_clean) == 0:
    print("FATAL ERROR: Filtering resulted in 0 rows.")
    sys.exit()

# --- 4. ADVANCED Feature Engineering (v3.1 - With Lag Fix) ---
def engineer_features(df, num_features, is_train=True):
    """Creates new, high-value features."""
    print("Engineering advanced features (v3.1)...")
    
    num_features_copy = num_features.copy()
    
    # Sort by time to create time-based features
    df = df.sort_values(by=['race_year', 'Rider_ID', 'seq'])
    
    # Feature 1: Tire Age (Improved)
    df['stint_id'] = (df['Tire_Compound'] != df.groupby('Rider_ID')['Tire_Compound'].shift()).astype(int).cumsum()
    df['tire_age'] = df.groupby(['Rider_ID', 'stint_id']).cumcount()
    if 'tire_age' not in num_features_copy:
        num_features_copy.append('tire_age')

    # Feature 2: Weather Interactions
    if 'weather' in df.columns:
        df['is_raining'] = df['weather'].astype(str).str.contains('Rain', case=False).fillna(0)
        if 'is_raining' not in num_features_copy:
            num_features_copy.append('is_raining')

    if 'Track_Temperature_Celsius' in df.columns:
        track_temp = pd.to_numeric(df['Track_Temperature_Celsius'], errors='coerce')
        ambient_temp = pd.to_numeric(df['Ambient_Temperature_Celsius'], errors='coerce')
        df['temp_diff'] = track_temp - ambient_temp
        if 'temp_diff' not in num_features_copy:
            num_features_copy.append('temp_diff')

        df['tire_age_x_track_temp'] = df['tire_age'] * track_temp
        if 'tire_age_x_track_temp' not in num_features_copy:
            num_features_copy.append('tire_age_x_track_temp')
            
    # Feature 3: Driver/Team Recent Form
    if 'points' in df.columns:
        df['driver_rolling_points'] = df.groupby('Rider_ID')['points'].rolling(window=5, min_periods=1).mean().reset_index(level=0, drop=True)
        if 'driver_rolling_points' not in num_features_copy:
            num_features_copy.append('driver_rolling_points')
            
    # --- Feature 4: Lag Feature (FIXED) ---
    if is_train:
        # If training, create it from the target
        df['previous_lap_time'] = df.groupby(['Rider_ID', 'stint_id'])[TARGET].shift(1)
        print("Engineered 'previous_lap_time' feature.")
    else:
        # If testing, create a placeholder NaN column
        df['previous_lap_time'] = np.nan
        print("Created 'previous_lap_time' placeholder for test set.")
    
    # Add the feature to the list *outside* the if/else
    if 'previous_lap_time' not in num_features_copy:
        num_features_copy.append('previous_lap_time')
    # ---------------------------------
    
    # Drop rows that have NaN from rolling/shift features (only for training)
    if is_train:
        # We now have two features that create NaNs on the first few rows
        df = df.dropna(subset=['driver_rolling_points', 'previous_lap_time'])

    return df, num_features_copy

df_featured, num_features_list = engineer_features(df_clean.copy(), NUMERIC_FEATURES, is_train=True)

# --- 5. Data Type Conversion (Training Data) ---
print("Converting data types and learning categories...")
for col in num_features_list:
    if col in df_featured.columns:
        df_featured[col] = pd.to_numeric(df_featured[col], errors='coerce') 
for col in CATEGORICAL_FEATURES:
    if col in df_featured.columns:
        df_featured[col] = df_featured[col].astype('category')
        CATEGORY_DTYPES[col] = df_featured[col].dtype
cat_features_list = [col for col in CATEGORICAL_FEATURES if col in df_featured.columns]
if TARGET not in df_featured.columns:
    print("FATAL ERROR: Target column not found.")
    sys.exit()

# --- 6. Create Data Splits ---
X = df_featured.drop(TARGET, axis=1)
y = df_featured[TARGET]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print(f"Total training samples: {len(X_train)}, Total test samples: {len(X_test)}")

# --- 7. Set up Preprocessing ---
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # Median is robust to outliers
    ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer, num_features_list)],
    remainder='drop'
)

# --- 8. Apply Preprocessing Manually ---
print("Applying preprocessing...")
preprocessor.fit(X_train[num_features_list])
def preprocess_data(df, processor, num_feats, cat_feats, cat_dtypes):
    df_numeric_scaled = pd.DataFrame(processor.transform(df[num_feats]), columns=num_feats, index=df.index)
    df_categorical = pd.DataFrame(index=df.index)
    for col in cat_feats:
        df_categorical[col] = pd.Categorical(df[col], categories=cat_dtypes[col].categories)
    df_final = pd.concat([df_numeric_scaled, df_categorical], axis=1)
    df_final[cat_feats] = df_final[cat_feats].apply(lambda x: x.cat.codes)
    return df_final

X_train_final = preprocess_data(X_train, preprocessor, num_features_list, cat_features_list, CATEGORY_DTYPES)
X_test_final = preprocess_data(X_test, preprocessor, num_features_list, cat_features_list, CATEGORY_DTYPES)
X_train_sub_final = preprocess_data(X_train_sub, preprocessor, num_features_list, cat_features_list, CATEGORY_DTYPES)
X_val_final = preprocess_data(X_val, preprocessor, num_features_list, cat_features_list, CATEGORY_DTYPES)
print("Preprocessing complete.")

# --- 9. Define the XGBoost Model and Tuning Parameters ---
print("Defining model and tuning search space...")
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=3000,
    early_stopping_rounds=100,
    random_state=42,
    tree_method='hist'
)

param_dist = {
    'learning_rate': uniform(0.01, 0.05),
    'max_depth': randint(6, 14),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4)
}

print("Starting RandomizedSearchCV... This will be slow!")
start_time = time.time()
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=30,
    cv=3,       
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,  
    verbose=1,  
    random_state=42
)

random_search.fit(
    X=X_train_sub_final, 
    y=y_train_sub,
    eval_set=[(X_val_final, y_val)],
    verbose=False
)
end_time = time.time()
print(f"✅ Tuner finished in {(end_time - start_time) / 60:.2f} minutes.")
print(f"Best parameters found: {random_search.best_params_}")
print(f"Best tuning score (RMSE): {-random_search.best_score_:.4f}")

# --- 10. Train the FINAL Model ---
print("\nTraining the final, best model on ALL training data...")
best_model = random_search.best_estimator_

best_model.fit(
    X_train_final, 
    y_train,
    eval_set=[(X_test_final, y_test)],
    verbose=False
)

# --- 11. Evaluate Your New Model on the Test Set ---
y_pred = best_model.predict(X_test_final)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"\n--- 🏁 FINAL Model Evaluation ---")
print(f"Final Test Set RMSE: {rmse:.4f} seconds")
print("This is the most reliable score for your new model.")

# --- 12. Load 'test.csv' for Final Predictions ---
print("\nLoading 'test.csv' for final predictions...")
df_test_raw = pd.read_csv('test.csv')
submission_ids = df_test_raw['id']

# --- 13. Apply the SAME Transformations to Test Data ---
print("Applying transformations to test data...")
# We now use is_train=False
df_test_engineered, test_num_feats = engineer_features(df_test_raw.copy(), NUMERIC_FEATURES, is_train=False)
for col in test_num_feats:
    if col in df_test_engineered.columns:
        df_test_engineered[col] = pd.to_numeric(df_test_engineered[col], errors='coerce')

# This call will now work
df_test_final = preprocess_data(
    df_test_engineered, 
    preprocessor, 
    num_features_list,
    cat_features_list, 
    CATEGORY_DTYPES
)

# --- 14. Make Final Predictions ---
print("Making predictions on test data...")
final_predictions = best_model.predict(df_test_final)

# --- 15. Create Submission File ---
print("Creating 'v12_tuned_submission.csv'...")
submission_df = pd.DataFrame({
    'id': submission_ids,
    'Lap_Time_Seconds': final_predictions
})
submission_df['Lap_Time_Seconds'] = submission_df['Lap_Time_Seconds'].fillna(y_train.mean())
submission_df['Lap_Time_Seconds'] = submission_df['Lap_Time_Seconds'].apply(lambda x: max(0, x))
submission_df.to_csv('v12_tuned_submission.csv', index=False)

print("\n✅ Done! Your new 'v12_tuned_submission.csv' is ready.")

--- Starting F1 Lap Time Model Training (v12, KeyError Fix) ---
Loading 'train.csv'...
Loaded 'train.csv' with 734002 rows.
Original data: 734002 rows
After all filters, 49074 rows remain for training.
Engineering advanced features (v3.1)...
Engineered 'previous_lap_time' feature.
Converting data types and learning categories...
Total training samples: 2552, Total test samples: 639
Applying preprocessing...
Preprocessing complete.
Defining model and tuning search space...
Starting RandomizedSearchCV... This will be slow!
Fitting 3 folds for each of 30 candidates, totalling 90 fits
✅ Tuner finished in 0.32 minutes.
Best parameters found: {'colsample_bytree': np.float64(0.7243929286862649), 'learning_rate': np.float64(0.026259166101337356), 'max_depth': 8, 'subsample': np.float64(0.8365191150830908)}
Best tuning score (RMSE): 11.5446

Training the final, best model on ALL training data...

--- 🏁 FINAL Model Evaluation ---
Final Test Set RMSE: 11.8274 seconds
This is the most reliable sco

In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import time
import sys
from scipy.stats import uniform, randint

print(f"--- Starting F1 Lap Time Model Training (v14, FAST-TUNE) ---")

# --- 1. Define Features & Target ---
TARGET = 'Lap_Time_Seconds' 
NUMERIC_FEATURES = [
    'Len_Circuit_inkm', 'Laps', 'Start_Position', 'Formula_Avg_Speed_kmh',
    'Humidity_%', 'Champ_Points', 'Champ_Position', 'race_year',
    'seq', 'position', 'points', 'Corners_in_Lap', 'Tire_Degradation_Factor_per_Lap',
    'Pit_Stop_Duration_Seconds', 'Ambient_Temperature_Celsius', 
    'Track_Temperature_Celsius', 'air', 'ground', 'starts', 'finishes',
    'with_points', 'podiums', 'wins'
]
CATEGORICAL_FEATURES = [
    'Rider_ID', 'Formula_category_x', 'Formula_Track_Condition',
    'Tire_Compound', 'Penalty', 'Session', 'Formula_shortname',
    'circuit_name', 'weather', 'track'
]
CATEGORY_DTYPES = {}

# --- 2. Load Your Training Data ---
try:
    print("Loading 'train.csv'...")
    df_raw = pd.read_csv('train.csv')
    print(f"Loaded 'train.csv' with {df_raw.shape[0]} rows.")
except FileNotFoundError:
    print("FATAL ERROR: 'train.csv' not found.")
    sys.exit()

# --- 3. Apply CORRECTED Data Filters ---
print(f"Original data: {len(df_raw)} rows")
df_clean = df_raw[df_raw['Session'] == 'Race'].copy()
df_clean = df_clean[df_clean['seq'] > 1].copy()
bad_laps = ['DNF', 'DNS', 'Ride Through']
df_clean = df_clean[~df_clean['Penalty'].isin(bad_laps)].copy()
print(f"After all filters, {len(df_clean)} rows remain for training.")
if len(df_clean) == 0:
    print("FATAL ERROR: Filtering resulted in 0 rows.")
    sys.exit()

# --- 4. ADVANCED Feature Engineering (v3.1 - With Lag Fix) ---
def engineer_features(df, num_features, is_train=True):
    """Creates new, high-value features."""
    print("Engineering advanced features (v3.1)...")
    
    num_features_copy = num_features.copy()
    
    df = df.sort_values(by=['race_year', 'Rider_ID', 'seq'])
    
    # Tire Age
    df['stint_id'] = (df['Tire_Compound'] != df.groupby('Rider_ID')['Tire_Compound'].shift()).astype(int).cumsum()
    df['tire_age'] = df.groupby(['Rider_ID', 'stint_id']).cumcount()
    if 'tire_age' not in num_features_copy:
        num_features_copy.append('tire_age')

    # Weather Interactions
    if 'weather' in df.columns:
        df['is_raining'] = df['weather'].astype(str).str.contains('Rain', case=False).fillna(0)
        if 'is_raining' not in num_features_copy:
            num_features_copy.append('is_raining')

    if 'Track_Temperature_Celsius' in df.columns:
        track_temp = pd.to_numeric(df['Track_Temperature_Celsius'], errors='coerce')
        ambient_temp = pd.to_numeric(df['Ambient_Temperature_Celsius'], errors='coerce')
        df['temp_diff'] = track_temp - ambient_temp
        if 'temp_diff' not in num_features_copy:
            num_features_copy.append('temp_diff')

        df['tire_age_x_track_temp'] = df['tire_age'] * track_temp
        if 'tire_age_x_track_temp' not in num_features_copy:
            num_features_copy.append('tire_age_x_track_temp')
            
    # Driver Form
    if 'points' in df.columns:
        df['driver_rolling_points'] = df.groupby('Rider_ID')['points'].rolling(window=5, min_periods=1).mean().reset_index(level=0, drop=True)
        if 'driver_rolling_points' not in num_features_copy:
            num_features_copy.append('driver_rolling_points')
            
    # Lag Feature
    if is_train:
        df['previous_lap_time'] = df.groupby(['Rider_ID', 'stint_id'])[TARGET].shift(1)
    else:
        df['previous_lap_time'] = np.nan
    if 'previous_lap_time' not in num_features_copy:
        num_features_copy.append('previous_lap_time')
    
    if is_train:
        df = df.dropna(subset=['driver_rolling_points', 'previous_lap_time'])

    return df, num_features_copy

df_featured, num_features_list = engineer_features(df_clean.copy(), NUMERIC_FEATURES, is_train=True)

# --- 5. Data Type Conversion (Training Data) ---
print("Converting data types and learning categories...")
for col in num_features_list:
    if col in df_featured.columns:
        df_featured[col] = pd.to_numeric(df_featured[col], errors='coerce') 
for col in CATEGORICAL_FEATURES:
    if col in df_featured.columns:
        df_featured[col] = df_featured[col].astype('category')
        CATEGORY_DTYPES[col] = df_featured[col].dtype
cat_features_list = [col for col in CATEGORICAL_FEATURES if col in df_featured.columns]
if TARGET not in df_featured.columns:
    print("FATAL ERROR: Target column not found.")
    sys.exit()

# --- 6. Create Data Splits ---
X = df_featured.drop(TARGET, axis=1)
y = df_featured[TARGET]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print(f"Total training samples: {len(X_train)}, Total test samples: {len(X_test)}")

# --- 7. Set up Preprocessing ---
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer, num_features_list)],
    remainder='drop'
)

# --- 8. Apply Preprocessing Manually ---
print("Applying preprocessing...")
preprocessor.fit(X_train[num_features_list])
def preprocess_data(df, processor, num_feats, cat_feats, cat_dtypes):
    df_numeric_scaled = pd.DataFrame(processor.transform(df[num_feats]), columns=num_feats, index=df.index)
    df_categorical = pd.DataFrame(index=df.index)
    for col in cat_feats:
        df_categorical[col] = pd.Categorical(df[col], categories=cat_dtypes[col].categories)
    df_final = pd.concat([df_numeric_scaled, df_categorical], axis=1)
    df_final[cat_feats] = df_final[cat_feats].apply(lambda x: x.cat.codes)
    return df_final

X_train_final = preprocess_data(X_train, preprocessor, num_features_list, cat_features_list, CATEGORY_DTYPES)
X_test_final = preprocess_data(X_test, preprocessor, num_features_list, cat_features_list, CATEGORY_DTYPES)
X_train_sub_final = preprocess_data(X_train_sub, preprocessor, num_features_list, cat_features_list, CATEGORY_DTYPES)
X_val_final = preprocess_data(X_val, preprocessor, num_features_list, cat_features_list, CATEGORY_DTYPES)
print("Preprocessing complete.")

# --- 9. Define the XGBoost Model and Tuning Parameters ---
print("Defining model and tuning search space...")
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=3000,
    early_stopping_rounds=100,
    random_state=42,
    tree_method='hist'
)

param_dist = {
    'learning_rate': uniform(0.01, 0.05),
    'max_depth': randint(6, 12),  # Slightly simpler depth range
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4)
}

print("Starting RandomizedSearchCV (n_iter=10)...")
start_time = time.time()
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=10,  # <-- FAST TUNING (was 30)
    cv=3,       
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,  
    verbose=1,  
    random_state=42
)

random_search.fit(
    X=X_train_sub_final, 
    y=y_train_sub,
    eval_set=[(X_val_final, y_val)],
    verbose=False
)
end_time = time.time()
print(f"✅ Tuner finished in {(end_time - start_time) / 60:.2f} minutes.")
print(f"Best parameters found: {random_search.best_params_}")
print(f"Best tuning score (RMSE): {-random_search.best_score_:.4f}")

# --- 10. Train the FINAL Model ---
print("\nTraining the final, best model on ALL training data...")
best_model = random_search.best_estimator_

best_model.fit(
    X_train_final, 
    y_train,
    eval_set=[(X_test_final, y_test)],
    verbose=False
)

# --- 11. Evaluate Your New Model on the Test Set ---
y_pred = best_model.predict(X_test_final)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"\n--- 🏁 FINAL Model Evaluation ---")
print(f"Final Test Set RMSE: {rmse:.4f} seconds")
print("This is the most reliable score for your new model.")

# --- 12. Load 'test.csv' for Final Predictions ---
print("\nLoading 'test.csv' for final predictions...")
df_test_raw = pd.read_csv('test.csv')
submission_ids = df_test_raw['id']

# --- 13. Apply the SAME Transformations to Test Data ---
print("Applying transformations to test data...")
df_test_engineered, test_num_feats = engineer_features(df_test_raw.copy(), NUMERIC_FEATURES, is_train=False)
for col in test_num_feats:
    if col in df_test_engineered.columns:
        df_test_engineered[col] = pd.to_numeric(df_test_engineered[col], errors='coerce')

df_test_final = preprocess_data(
    df_test_engineered, 
    preprocessor, 
    num_features_list,
    cat_features_list, 
    CATEGORY_DTYPES
)

# --- 14. Make Final Predictions ---
print("Making predictions on test data...")
final_predictions = best_model.predict(df_test_final)

# --- 15. Create Submission File ---
print("Creating 'v14_fast_submission.csv'...")
submission_df = pd.DataFrame({
    'id': submission_ids,
    'Lap_Time_Seconds': final_predictions
})
submission_df['Lap_Time_Seconds'] = submission_df['Lap_Time_Seconds'].fillna(y_train.mean())
submission_df['Lap_Time_Seconds'] = submission_df['Lap_Time_Seconds'].apply(lambda x: max(0, x))
submission_df.to_csv('v14_fast_submission.csv', index=False)

print("\n✅ Done! Your new 'v14_fast_submission.csv' is ready.")

--- Starting F1 Lap Time Model Training (v14, FAST-TUNE) ---
Loading 'train.csv'...
Loaded 'train.csv' with 734002 rows.
Original data: 734002 rows
After all filters, 49074 rows remain for training.
Engineering advanced features (v3.1)...
Converting data types and learning categories...
Total training samples: 2552, Total test samples: 639
Applying preprocessing...
Preprocessing complete.
Defining model and tuning search space...
Starting RandomizedSearchCV (n_iter=10)...
Fitting 3 folds for each of 10 candidates, totalling 30 fits
✅ Tuner finished in 0.11 minutes.
Best parameters found: {'colsample_bytree': np.float64(0.7727780074568463), 'learning_rate': np.float64(0.024561457009902095), 'max_depth': 8, 'subsample': np.float64(0.7599443886861021)}
Best tuning score (RMSE): 11.5574

Training the final, best model on ALL training data...

--- 🏁 FINAL Model Evaluation ---
Final Test Set RMSE: 11.8145 seconds
This is the most reliable score for your new model.

Loading 'test.csv' for fin